In [1]:
import os
from dotenv import load_dotenv


load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

YOUTUBE_URL = "https://www.youtube.com/watch?v=9UxAUryUKXM"

In [2]:
from langchain_anthropic import ChatAnthropic
model = ChatAnthropic(model='claude-3-sonnet-20240229')

In [3]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [4]:
from langchain.prompts import ChatPromptTemplate
template = """ Answer the question succinctly using the context. If You can't answer the question, reply "I don't know". Context: {context} Question: {question} """
prompt = ChatPromptTemplate.from_template(template)

In [6]:
import os
import whisper
import yt_dlp
import time

def download_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [],
        'outtmpl': '%(id)s.%(ext)s',
        'progress_hooks': [lambda d: print(f'Downloading: {d["_percent_str"]}')]
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        return ydl.prepare_filename(info)

def transcribe_audio(filename):
    print("Loading Whisper model...")
    model = whisper.load_model("tiny")
    print("Transcribing audio...")
    start_time = time.time()
    result = model.transcribe(filename, fp16=False)
    end_time = time.time()
    print(f"Transcription took {end_time - start_time:.2f} seconds")
    return result["text"].strip()

if not os.path.exists("transcription.txt"):
    YOUTUBE_URL = "https://www.youtube.com/watch?v=9UxAUryUKXM"  # Replace with your YouTube URL
    
    print("Downloading audio...")
    filename = download_audio(YOUTUBE_URL)
    
    transcription = transcribe_audio(filename)
    
    with open("transcription.txt", "w", encoding="utf-8") as file:
        file.write(transcription)
    
    os.remove(filename)

print("Transcription complete. Results saved in 'transcription.txt'")

[youtube] Extracting URL: https://www.youtube.com/watch?v=9UxAUryUKXM
[youtube] 9UxAUryUKXM: Downloading webpage
[youtube] 9UxAUryUKXM: Downloading ios player API JSON
[youtube] 9UxAUryUKXM: Downloading tv player API JSON
[youtube] 9UxAUryUKXM: Downloading m3u8 information
[info] 9UxAUryUKXM: Downloading 1 format(s): 251
[download] 9UxAUryUKXM.webm has already been downloaded
[download] 100% of   42.52MiBDownloading: 100.0%

Loading Whisper model...


c:\Users\Brand\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

Transcribing audio...
Transcription took 276.65 seconds
Transcription complete. Results saved in 'transcription.txt'


In [10]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcription.txt", encoding="utf-8")
text_documents = loader.load()

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [12]:
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()


In [13]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_pinecone import PineconeVectorStore

index_name = "youtube-rag-index"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)

c:\Users\Brand\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [14]:
setup_context = RunnableParallel(context=pinecone.as_retriever(), question=RunnablePassthrough())
chain = setup_context | prompt | model | parser

In [16]:
chain.invoke("Does the speaker mention buying more Bitcoin at all?")

'No, the speaker does not mention buying more Bitcoin at all in the given context.'